# Tensorflow Object Detection API and AWS Sagemaker

In this notebook, you will train and evaluate different models using the [Tensorflow Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/) and [AWS Sagemaker](https://aws.amazon.com/sagemaker/). 

If you ever feel stuck, you can refer to this [tutorial](https://aws.amazon.com/blogs/machine-learning/training-and-deploying-models-using-tensorflow-2-with-the-object-detection-api-on-amazon-sagemaker/).

## Dataset

We are using the [Waymo Open Dataset](https://waymo.com/open/) for this project. The dataset has already been exported using the tfrecords format. The files have been created following the format described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). You can find data stored on [AWS S3](https://aws.amazon.com/s3/), AWS Object Storage. The images are saved with a resolution of 640x640.

In [1]:
%%capture
%pip install tensorflow_io sagemaker -U

Save the IAM role in a variable called `role`. This would be useful when training the model.

In [2]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::642356378541:role/service-role/AmazonSageMaker-ExecutionRole-20231009T075267


In [4]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
        'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://object-detection-project/logs/'

## Container

To train the model, you will first need to build a [docker](https://www.docker.com/) container with all the dependencies required by the TF Object Detection API. The code below does the following:
* clone the Tensorflow models repository
* get the exporter and training scripts from the the repository
* build the docker image and push it 
* print the container name

In [5]:
%%bash

# clone the repo and get the scripts
git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Object Detection GitHub repository
cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
cp docker/models/research/object_detection/model_main_tf2.py source_dir

Cloning into 'docker/models'...


In [ ]:
# build and push the docker image. This code can be commented after being ran once.
# This will take around 10 mins.
image_name = 'tf2-object-detection'
!sh ./docker/build_and_push.sh $image_name

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Building image with name tf2-object-detection
Sending build context to Docker daemon  741.8MB
Step 1/17 : FROM tensorflow/tensorflow:2.9.0-gpu
 ---> c8d9ee2a0ff4
Step 2/17 : ARG DEBIAN_FRONTEND=noninteractive
 ---> Running in 7a7f10702725
Removing intermediate container 7a7f10702725
 ---> 1fd38c0c6e2a
Step 3/17 : RUN rm /etc/apt/sources.list.d/cuda.list
 ---> Running in 3d29a580934c
Removing intermediate container 3d29a580934c
 ---> 8f5f69bf0729
Step 4/17 : RUN apt-key del 7fa2af80
 ---> Running in 0a43d2ebd3a4
OK
Removing intermediate container 0a43d2ebd3a4
 ---> 0e5000cc2a26
Step 5/17 : RUN apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/3bf863cc.pub
 ---> Running in 6d0c2fe2c255
Executing: /tmp

Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 dirmngr amd64 2.2.19-3ubuntu2.2 [330 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gpg-wks-server amd64 2.2.19-3ubuntu2.2 [90.2 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gnupg-utils amd64 2.2.19-3ubuntu2.2 [481 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gpg-agent amd64 2.2.19-3ubuntu2.2 [232 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gpg amd64 2.2.19-3ubuntu2.2 [482 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gpgconf amd64 2.2.19-3ubuntu2.2 [124 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gnupg-l10n all 2.2.19-3ubuntu2.2 [51.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gnupg all 2.2.19-3ubuntu2.2 [259 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gpgsm amd64 2.2.19-3ubuntu2.2 [217 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-up

Get:88 http://archive.ubuntu.com/ubuntu focal/main amd64 libxcb1-dev amd64 1.14-2 [80.5 kB]
Get:89 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libx11-dev amd64 2:1.6.9-2ubuntu1.6 [648 kB]
Get:90 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libglx-dev amd64 1.3.2-1~ubuntu0.20.04.2 [14.0 kB]
Get:91 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgl-dev amd64 1.3.2-1~ubuntu0.20.04.2 [97.8 kB]
Get:92 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libegl-dev amd64 1.3.2-1~ubuntu0.20.04.2 [17.2 kB]
Get:93 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libjbig0 amd64 2.1-3.1ubuntu0.20.04.1 [27.3 kB]
Get:94 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libwebp6 amd64 0.6.1-2ubuntu0.20.04.3 [185 kB]
Get:95 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libtiff5 amd64 4.1.0+git191117-2ubuntu0.20.04.11 [164 kB]
Get:96 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgdk-pixbuf2.0-common all 2.40.0+dfs

Selecting previously unselected package libxau6:amd64.
Preparing to unpack .../010-libxau6_1%3a1.0.9-0ubuntu1_amd64.deb ...
Unpacking libxau6:amd64 (1:1.0.9-0ubuntu1) ...
Selecting previously unselected package libxdmcp6:amd64.
Preparing to unpack .../011-libxdmcp6_1%3a1.1.3-0ubuntu1_amd64.deb ...
Unpacking libxdmcp6:amd64 (1:1.1.3-0ubuntu1) ...
Selecting previously unselected package libxcb1:amd64.
Preparing to unpack .../012-libxcb1_1.14-2_amd64.deb ...
Unpacking libxcb1:amd64 (1.14-2) ...
Selecting previously unselected package libx11-data.
Preparing to unpack .../013-libx11-data_2%3a1.6.9-2ubuntu1.6_all.deb ...
Unpacking libx11-data (2:1.6.9-2ubuntu1.6) ...
Selecting previously unselected package libx11-6:amd64.
Preparing to unpack .../014-libx11-6_2%3a1.6.9-2ubuntu1.6_amd64.deb ...
Unpacking libx11-6:amd64 (2:1.6.9-2ubuntu1.6) ...
Selecting previously unselected package libxext6:amd64.
Preparing to unpack .../015-libxext6_2%3a1.3.4-0ubuntu1_amd64.deb ...
Unpacking libxext6:amd64 (

Selecting previously unselected package libegl1:amd64.
Preparing to unpack .../057-libegl1_1.3.2-1~ubuntu0.20.04.2_amd64.deb ...
Unpacking libegl1:amd64 (1.3.2-1~ubuntu0.20.04.2) ...
Selecting previously unselected package libxcb-glx0:amd64.
Preparing to unpack .../058-libxcb-glx0_1.14-2_amd64.deb ...
Unpacking libxcb-glx0:amd64 (1.14-2) ...
Selecting previously unselected package libxfixes3:amd64.
Preparing to unpack .../059-libxfixes3_1%3a5.0.3-2_amd64.deb ...
Unpacking libxfixes3:amd64 (1:5.0.3-2) ...
Selecting previously unselected package libxxf86vm1:amd64.
Preparing to unpack .../060-libxxf86vm1_1%3a1.1.4-1build1_amd64.deb ...
Unpacking libxxf86vm1:amd64 (1:1.1.4-1build1) ...
Selecting previously unselected package libllvm12:amd64.
Preparing to unpack .../061-libllvm12_1%3a12.0.0-3ubuntu1~20.04.5_amd64.deb ...
Unpacking libllvm12:amd64 (1:12.0.0-3ubuntu1~20.04.5) ...
Selecting previously unselected package libsensors-config.
Preparing to unpack .../062-libsensors-config_1%3a3.6.0

...

  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.25.0
    Uninstalling tensorflow-io-gcs-filesystem-0.25.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.25.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
      Successfully uninstalled tensorboard-data-server-0.6.1
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.1/699.1 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.3/249.3 kB 24.7 MB/s eta 0:00:00
  Created wheel for sagemaker-training: filename=sagemaker_training-4.7.4-cp38-cp38-linux_x86_64.whl size=93696 sha256=d4a4d6a130ee6081da6eeca5762bb058eee5c0985cb097c8a95e94720270b958
  Stored in director

60d7410: Pushing  1.178GB/3.407GB

60d7410: Pushing   2.86GB/3.407GBPushing  1.197GB/3.407GBPushing  1.456GB/3.407GBPushing  1.773GB/3.407GB

60d7410: Pushed   3.444GB/3.407GBPushing  2.973GB/3.407GBPushing  3.249GB/3.407GB20231226144956: digest: sha256:4b12463618f4e2e2eb8a428031a1072ef4de59c1578c129048e247e5e67ac6c0 size: 5782
Saving ECR image URI into ecr_image_fullname.txt


To verify that the image was correctly pushed to the [Elastic Container Registry](https://aws.amazon.com/ecr/), you can look at it in the AWS webapp. For example, below you can see that three different images have been pushed to ECR. You should only see one, called `tf2-object-detection`.
![ECR Example](../data/example_ecr.png)


In [ ]:
# display the container name
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

642356378541.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection:20231226144956


## Pre-trained model from model zoo

As often, we are not training from scratch and we will be using a pretrained model from the TF Object Detection model zoo. You can find pretrained checkpoints [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Because your time is limited for this project, we recommend to only experiment with the following models:
* SSD MobileNet V2 FPNLite 640x640	
* SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	
* Faster R-CNN ResNet50 V1 640x640	
* EfficientDet D1 640x640	
* Faster R-CNN ResNet152 V1 640x640	

In the code below, the EfficientDet D1 model is downloaded and extracted. This code should be ajusted if you were to experiment with other architectures.

In [ ]:
%%bash
mkdir /tmp/checkpoint
mkdir source_dir/checkpoint
wget -O /tmp/faster_rcnn_resnet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz
tar -zxvf /tmp/faster_rcnn_resnet.tar.gz --strip-components 2 --directory source_dir/checkpoint faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/checkpoint

mkdir: cannot create directory ‘/tmp/checkpoint’: File exists
mkdir: cannot create directory ‘source_dir/checkpoint’: File exists
--2023-12-26 15:00:36--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.253.122.207, 172.253.63.207, 142.251.111.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|172.253.122.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 470656289 (449M) [application/x-tar]
Saving to: ‘/tmp/faster_rcnn_resnet.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 9.13M 49s
    50K .......... .......... .......... .......... ..........  0% 18.2M 37s
   100K .......... .......... .......... .......... ..........  0% 16.9M 33s
   150K .......... .......... .......... .......... ..........  0% 12.7M 34s
   200K .......... .......... .......... .......... ..........  

  4950K .......... .......... .......... .......... ..........  1% 82.9M 9s
  5000K .......... .......... .......... .......... ..........  1%  197M 9s
  5050K .......... .......... .......... .......... ..........  1% 72.5M 9s
  5100K .......... .......... .......... .......... ..........  1% 73.1M 9s
  5150K .......... .......... .......... .......... ..........  1% 80.2M 9s
  5200K .......... .......... .......... .......... ..........  1%  339M 9s
  5250K .......... .......... .......... .......... ..........  1% 66.9M 9s
  5300K .......... .......... .......... .......... ..........  1% 83.9M 9s
  5350K .......... .......... .......... .......... ..........  1% 61.7M 9s
  5400K .......... .......... .......... .......... ..........  1%  100M 9s
  5450K .......... .......... .......... .......... ..........  1%  292M 9s
  5500K .......... .......... .......... .......... ..........  1% 81.3M 9s
  5550K .......... .......... .......... .......... ..........  1% 72.8M 9s
  5600K ....

 10350K .......... .......... .......... .......... ..........  2% 79.6M 10s
 10400K .......... .......... .......... .......... ..........  2%  134M 10s
 10450K .......... .......... .......... .......... ..........  2%  347M 10s
 10500K .......... .......... .......... .......... ..........  2%  106M 10s
 10550K .......... .......... .......... .......... ..........  2%  102M 10s
 10600K .......... .......... .......... .......... ..........  2%  111M 10s
 10650K .......... .......... .......... .......... ..........  2%  117M 10s
 10700K .......... .......... .......... .......... ..........  2%  186M 10s
 10750K .......... .......... .......... .......... ..........  2% 91.6M 10s
 10800K .......... .......... .......... .......... ..........  2%  149M 10s
 10850K .......... .......... .......... .......... ..........  2% 33.5M 10s
 10900K .......... .......... .......... .......... ..........  2%  294M 10s
 10950K .......... .......... .......... .......... ..........  2%  317M 10s

 15750K .......... .......... .......... .......... ..........  3%  326M 8s
 15800K .......... .......... .......... .......... ..........  3%  343M 8s
 15850K .......... .......... .......... .......... ..........  3%  302M 8s
 15900K .......... .......... .......... .......... ..........  3% 70.3M 8s
 15950K .......... .......... .......... .......... ..........  3%  347M 8s
 16000K .......... .......... .......... .......... ..........  3%  353M 8s
 16050K .......... .......... .......... .......... ..........  3% 68.7M 8s
 16100K .......... .......... .......... .......... ..........  3%  357M 8s
 16150K .......... .......... .......... .......... ..........  3%  279M 8s
 16200K .......... .......... .......... .......... ..........  3% 69.0M 8s
 16250K .......... .......... .......... .......... ..........  3%  304M 8s
 16300K .......... .......... .......... .......... ..........  3%  351M 8s
 16350K .......... .......... .......... .......... ..........  3%  557K 10s
 16400K ...

 21150K .......... .......... .......... .......... ..........  4%  285M 9s
 21200K .......... .......... .......... .......... ..........  4%  235M 9s
 21250K .......... .......... .......... .......... ..........  4%  213M 9s
 21300K .......... .......... .......... .......... ..........  4%  326M 9s
 21350K .......... .......... .......... .......... ..........  4%  263M 9s
 21400K .......... .......... .......... .......... ..........  4%  292M 9s
 21450K .......... .......... .......... .......... ..........  4%  331M 9s
 21500K .......... .......... .......... .......... ..........  4%  290M 9s
 21550K .......... .......... .......... .......... ..........  4%  301M 9s
 21600K .......... .......... .......... .......... ..........  4%  332M 9s
 21650K .......... .......... .......... .......... ..........  4%  322M 8s
 21700K .......... .......... .......... .......... ..........  4%  278M 8s
 21750K .......... .......... .......... .......... ..........  4%  296M 8s
 21800K ....

...

420750K .......... .......... .......... .......... .......... 91% 10.8M 1s
420800K .......... .......... .......... .......... .......... 91%  299M 1s
420850K .......... .......... .......... .......... .......... 91%  365M 1s
420900K .......... .......... .......... .......... .......... 91%  366M 1s
420950K .......... .......... .......... .......... .......... 91%  371M 1s
421000K .......... .......... .......... .......... .......... 91%  282M 1s
421050K .......... .......... .......... .......... .......... 91%  363M 1s
421100K .......... .......... .......... .......... .......... 91%  367M 1s
421150K .......... .......... .......... .......... .......... 91%  375M 1s
421200K .......... .......... .......... .......... .......... 91%  303M 1s
421250K .......... .......... .......... .......... .......... 91%  373M 1s
421300K .......... .......... .......... .......... .......... 91%  361M 1s
421350K .......... .......... .......... .......... .......... 91%  366M 1s
421400K ....

426150K .......... .......... .......... .......... .......... 92%  373M 1s
426200K .......... .......... .......... .......... .......... 92%  325M 0s
426250K .......... .......... .......... .......... .......... 92%  369M 0s
426300K .......... .......... .......... .......... .......... 92%  384M 0s
426350K .......... .......... .......... .......... .......... 92% 6.66M 0s
426400K .......... .......... .......... .......... .......... 92%  302M 0s
426450K .......... .......... .......... .......... .......... 92%  364M 0s
426500K .......... .......... .......... .......... .......... 92%  344M 0s
426550K .......... .......... .......... .......... .......... 92%  363M 0s
426600K .......... .......... .......... .......... .......... 92%  318M 0s
426650K .......... .......... .......... .......... .......... 92%  285M 0s
426700K .......... .......... .......... .......... .......... 92%  363M 0s
426750K .......... .......... .......... .......... .......... 92%  367M 0s
426800K ....

431550K .......... .......... .......... .......... .......... 93%  370M 0s
431600K .......... .......... .......... .......... .......... 93%  302M 0s
431650K .......... .......... .......... .......... .......... 93%  372M 0s
431700K .......... .......... .......... .......... .......... 93%  371M 0s
431750K .......... .......... .......... .......... .......... 93%  378M 0s
431800K .......... .......... .......... .......... .......... 93%  321M 0s
431850K .......... .......... .......... .......... .......... 93%  366M 0s
431900K .......... .......... .......... .......... .......... 93%  362M 0s
431950K .......... .......... .......... .......... .......... 93%  296M 0s
432000K .......... .......... .......... .......... .......... 94%  320M 0s
432050K .......... .......... .......... .......... .......... 94%  372M 0s
432100K .......... .......... .......... .......... .......... 94%  369M 0s
432150K .......... .......... .......... .......... .......... 94%  328M 0s
432200K ....

436950K .......... .......... .......... .......... .......... 95%  361M 0s
437000K .......... .......... .......... .......... .......... 95%  331M 0s
437050K .......... .......... .......... .......... .......... 95%  372M 0s
437100K .......... .......... .......... .......... .......... 95%  363M 0s
437150K .......... .......... .......... .......... .......... 95%  333M 0s
437200K .......... .......... .......... .......... .......... 95%  295M 0s
437250K .......... .......... .......... .......... .......... 95%  377M 0s
437300K .......... .......... .......... .......... .......... 95%  283M 0s
437350K .......... .......... .......... .......... .......... 95%  196M 0s
437400K .......... .......... .......... .......... .......... 95%  164M 0s
437450K .......... .......... .......... .......... .......... 95%  155M 0s
437500K .......... .......... .......... .......... .......... 95%  270M 0s
437550K .......... .......... .......... .......... .......... 95%  157M 0s
437600K ....

faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/checkpoint/checkpoint
faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.index


## Edit pipeline.config file

The [`pipeline.config`](source_dir/pipeline.config) in the `source_dir` folder should be updated when you experiment with different models. The different config files are available [here](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2).

>Note: The provided `pipeline.config` file works well with the `EfficientDet` model. You would need to modify it when working with other models.

## Launch Training Job

Now that we have a dataset, a docker image and some pretrained model weights, we can launch the training job. To do so, we create a [Sagemaker Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html), where we indicate the container name, name of the config file, number of training steps etc.

The `run_training.sh` script does the following:
* train the model for `num_train_steps` 
* evaluate over the val dataset
* export the model

Different metrics will be displayed during the evaluation phase, including the mean average precision. These metrics can be used to quantify your model performances and compare over the different iterations.

You can also monitor the training progress by navigating to **Training -> Training Jobs** from the Amazon Sagemaker dashboard in the Web UI.

In [ ]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point='run_training.sh',
    source_dir='source_dir/',
    hyperparameters={
        "model_dir":"/opt/training",        
        "pipeline_config_path": "pipeline.config",
        "num_train_steps": "2000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='tf2-object-detection'
)

estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: tf2-object-detection-2023-12-26-15-00-51-776


2023-12-26 15:01:12 Starting - Starting the training job...
2023-12-26 15:01:39 Starting - Preparing the instances for training.........
2023-12-26 15:02:56 Downloading - Downloading input data...
2023-12-26 15:03:27 Downloading - Downloading the training image.........
2023-12-26 15:05:12 Training - Training image download completed. Training in progress......2023-12-26 15:05:45,382 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-12-26 15:05:45,432 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-12-26 15:05:45,478 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-12-26 15:05:45,497 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train",
        "val": "/opt/ml/input/data/val"
    },
    "current_host": "algo-1",
    "cur

W1226 15:05:59.383595 140612546271040 model_lib_v2.py:1089] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I1226 15:05:59.383829 140612546271040 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1226 15:05:59.383939 140612546271040 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I1226 15:05:59.384055 140612546271040 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W1226 15:05:59.384232 140612546271040 model_lib_v2.py:1106] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
Traceback (most recent call last):
  File "model_main_tf2.py", line 114, in <module>
    tf.compat.v1.app.run()
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/platform/app.py", line 36,

UnexpectedStatusException: Error for Training job tf2-object-detection-2023-12-26-15-00-51-776: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage ""
Command "/bin/sh -c ./run_training.sh --model_dir /opt/training --num_train_steps 2000 --pipeline_config_path pipeline.config --sample_1_of_n_eval_examples 1", exit code: 1

You should be able to see your model training in the AWS webapp as shown below:
![ECR Example](../data/example_trainings.png)


## Improve on the intial model

Most likely, this initial experiment did not yield optimal results. However, you can make multiple changes to the `pipeline.config` file to improve this model. One obvious change consists in improving the data augmentation strategy. The [`preprocessor.proto`](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto) file contains the different data augmentation method available in the Tf Object Detection API. Justify your choices of augmentations in the writeup.

Keep in mind that the following are also available:
* experiment with the optimizer: type of optimizer, learning rate, scheduler etc
* experiment with the architecture. The Tf Object Detection API model zoo offers many architectures. Keep in mind that the pipeline.config file is unique for each architecture and you will have to edit it.
* visualize results on the test frames using the `2_deploy_model` notebook available in this repository.

In the cell below, write down all the different approaches you have experimented with, why you have chosen them and what you would have done if you had more time and resources. Justify your choices using the tensorboard visualizations (take screenshots and insert them in your writeup), the metrics on the evaluation set and the generated animation you have created with [this tool](../2_run_inference/2_deploy_model.ipynb).

In [ ]:
# your writeup goes here.